In [1]:
from unittest.mock import Mock, patch, call

import pytest
from unittest.mock import MagicMock

from fastlane_bot import Bot, Config
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3,  CarbonV1, BancorV3
from fastlane_bot.events.managers.manager import Manager
Base = None
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC

print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *

#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)


ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.2 [requirements >= 3.0 is met]


c:\Users\Kveen\PycharmProjects\fastlane-bot\venv_11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

with open("fastlane_bot/data/event_test_data.json", "r") as f:
    event_data = json.load(f)

with open("fastlane_bot/data/test_pool_data.json", "r") as f:
    pool_data = json.load(f)

In [3]:
from fastlane_bot.events.utils import handle_static_pools_update, get_static_data

# Create mock instances for all required parameters
cfg = Config.new(config=Config.CONFIG_MAINNET)

(
    static_pool_data,
    tokens,
    uniswap_v2_event_mappings,
    uniswap_v3_event_mappings,
    solidly_v2_event_mappings,
) = get_static_data(
    cfg, ['bancor_v3', 'carbon_v1', 'uniswap_v2', 'uniswap_v3'], "ethereum",'static_pool_data'
)
# create manager instance for all tests
manager = Manager(web3=cfg.w3, 
                  w3_async=cfg.w3_async,
                  cfg=cfg, 
                  pool_data=pool_data, 
                  alchemy_max_block_fetch=20, 
                  SUPPORTED_EXCHANGES=['bancor_v3', 'carbon_v1', 'uniswap_v2', 'uniswap_v3'],
                  forked_exchanges=['uniswap_v2', 'uniswap_v3'],
                  uniswap_v2_event_mappings=uniswap_v2_event_mappings,
                  uniswap_v3_event_mappings=uniswap_v3_event_mappings,
                  solidly_v2_event_mappings=solidly_v2_event_mappings)

handle_static_pools_update(manager)

2024-01-04 14:04:51,247 [fastlane:INFO] - 
2024-01-04 14:04:51,248 [fastlane:INFO] - **********************************************
2024-01-04 14:04:51,248 [fastlane:INFO] - The logging path is set to: logs/20240104-140451\bot.log
2024-01-04 14:04:51,249 [fastlane:INFO] - **********************************************
2024-01-04 14:04:51,249 [fastlane:INFO] - 
2024-01-04 14:04:57,533 [fastlane:INFO] - [events.utils.handle_static_pools_update] Adding 286307 uniswap_v2 pools to static pools
2024-01-04 14:04:57,541 [fastlane:INFO] - [events.utils.handle_static_pools_update] Adding 17983 uniswap_v3 pools to static pools


## test_update_from_event_uniswap_v2

In [4]:
event = event_data['uniswap_v2_event']

assert event['args']['reserve0'] != [pool['tkn0_balance'] for pool in manager.pool_data if pool['address'] == event['address']][0]

manager.update_from_event(event)

assert event['address'] in [pool['address'] for pool in manager.pool_data]
assert event['args']['reserve0'] == [pool['tkn0_balance'] for pool in manager.pool_data if pool['address'] == event['address']][0]

## test_update_from_event_uniswap_v3

In [5]:
event = event_data['uniswap_v3_event']

assert event['args']['liquidity'] != [pool['liquidity'] for pool in manager.pool_data if pool['address'] == event['address']][0]

manager.update_from_event(event)

assert event['address'] in [pool['address'] for pool in manager.pool_data]
assert event['args']['liquidity'] == [pool['liquidity'] for pool in manager.pool_data if pool['address'] == event['address']][0]

## test_update_from_event_carbon_v1_update

In [6]:
event_create_for_update = event_data['carbon_v1_event_create_for_update']
event = event_data['carbon_v1_event_update']

manager.update_from_event(event_create_for_update)
pools_to_add_from_contracts = [event[2]['args']['id'] for event in manager.pools_to_add_from_contracts]

assert event['args']['id'] in pools_to_add_from_contracts


## test_update_from_event_carbon_v1_create


In [7]:
event = event_data['carbon_v1_event_create']
manager.pool_data = [pool for pool in manager.pool_data if pool['cid'] != event['args']['id']]
assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]

manager.update_from_event(event)

assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]

## test_update_from_event_carbon_v1_delete

In [8]:
event = event_data['carbon_v1_event_create']
manager.pool_data = [pool for pool in manager.pool_data if pool['cid'] != event['args']['id']]
assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]

manager.update_from_event(event)

assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]

event['event'] = 'StrategyDeleted'

manager.update_from_event(event)

assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]